In [1]:
import sys
!{sys.executable} -m pip install geocoder
!{sys.executable} -m pip install folium

print('Packages installed.')

     |████████████████████████████████| 102kB 18.4MB/s ta 0:00:01
Packages installed.


In [2]:
pip install BeautifulSoup4

     |████████████████████████████████| 122kB 17.0MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install Nominatim

  Stored in directory: /home/jupyterlab/.cache/pip/wheels/d5/66/ed/e7476981dc30210b6b5ce7c25b054e8db35d44fdd2198003d4
Successfully built Nominatim
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install geopy

     |████████████████████████████████| 112kB 18.0MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
import requests
import math
import folium
import geocoder
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import matplotlib.cm as cm  # Matplotlib and associated plotting modules
import matplotlib.colors as colors
from matplotlib import pyplot as plt
from sklearn import metrics 
from scipy.spatial.distance import cdist
from sklearn.cluster import KMeans # KMeans for clustering
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
import json
import seaborn as sns
sns.set_style('darkgrid')

In [6]:
url = 'https://en.wikipedia.org/wiki/Boroughs_of_Amsterdam'

In [7]:
table = pd.read_html(url)

# Boroughs
print(table[1])  # index 1 returns the 2nd table from the web page

                   Borough       Area  Population Population density  \
0         Centrum (Centre)   8.04 km²       86422         13,748/km²   
1            Noord (North)  49.01 km²       94766          2,269/km²   
2     Nieuw-West(New West)  32.38 km²      151677          4,478/km²   
3              Oost (East)  30.56 km²      135767          7,635/km²   
4                     West   9.89 km²      143842         15,252/km²   
5  Westpoort(West Gateway)     10 km²         192             10/km²   
6             Zuid (South)  17.41 km²      144432          9,349/km²   
7      Zuidoost(Southeast)  22.08 km²       87854          4,391/km²   

   Location (in green)                                     Neighbourhoods  
0                  NaN  Binnenstad, Grachtengordel, Haarlemmerbuurt, J...  
1                  NaN  Banne Buiksloot, Buiksloot, Buikslotermeer, Fl...  
2                  NaN  Geuzenveld, Nieuw Sloten, Oostoever, Osdorp, O...  
3                  NaN  IJburg, Indische Buurt,

In [8]:
# Create Borough data frame - bor_df
bor_df = pd.DataFrame(table[1])
print(bor_df)

                   Borough       Area  Population Population density  \
0         Centrum (Centre)   8.04 km²       86422         13,748/km²   
1            Noord (North)  49.01 km²       94766          2,269/km²   
2     Nieuw-West(New West)  32.38 km²      151677          4,478/km²   
3              Oost (East)  30.56 km²      135767          7,635/km²   
4                     West   9.89 km²      143842         15,252/km²   
5  Westpoort(West Gateway)     10 km²         192             10/km²   
6             Zuid (South)  17.41 km²      144432          9,349/km²   
7      Zuidoost(Southeast)  22.08 km²       87854          4,391/km²   

   Location (in green)                                     Neighbourhoods  
0                  NaN  Binnenstad, Grachtengordel, Haarlemmerbuurt, J...  
1                  NaN  Banne Buiksloot, Buiksloot, Buikslotermeer, Fl...  
2                  NaN  Geuzenveld, Nieuw Sloten, Oostoever, Osdorp, O...  
3                  NaN  IJburg, Indische Buurt,

In [9]:
# Dropping unnecessary columns, we keep only Boroughs and Population
bor_df = bor_df.drop(['Area', 'Population density', 'Location (in green)'],1)
bor_df.replace({'Centrum (Centre)':'Centrum', 'Noord (North)':'Noord', 'Nieuw-West(New West)':'Nieuw-West',
                    'Oost (East)':'Oost', 'Westpoort(West Gateway)':'Westpoort', 'Zuid (South)':'Zuid', 
                    'Zuidoost(Southeast)':'Zuidoost'}, inplace=True)

print(bor_df)
bor_df.to_csv('Boroughs.csv')

      Borough  Population                                     Neighbourhoods
0     Centrum       86422  Binnenstad, Grachtengordel, Haarlemmerbuurt, J...
1       Noord       94766  Banne Buiksloot, Buiksloot, Buikslotermeer, Fl...
2  Nieuw-West      151677  Geuzenveld, Nieuw Sloten, Oostoever, Osdorp, O...
3        Oost      135767  IJburg, Indische Buurt, Eastern Docklands, Oud...
4        West      143842  Frederik Hendrikbuurt, Houthaven, Spaarndammer...
5   Westpoort         192                                          Westpoort
6        Zuid      144432  Apollobuurt, Buitenveldert, Hoofddorppleinbuur...
7    Zuidoost       87854    Bijlmermeer, Venserpolder, Gaasperdam, Driemond


In [10]:
# Neighborhoods
print(table[3])  # index 3 returns the 4th table from the web page

                       vteNeighborhoods of Amsterdam  \
0                                            Centrum   
1                                         Nieuw-West   
2                                              Noord   
3                                               Oost   
4                                               West   
5                                               Zuid   
6                                           Zuidoost   
7                                          Westpoort   
8  Former boroughs: De Baarsjes - Geuzenveld-Slot...   

                     vteNeighborhoods of Amsterdam.1  \
0  Binnenstad (Oude Zijde - Nieuwe Zijde) Grachte...   
1  Geuzenveld (De Eendracht) Nieuw Sloten Oostoev...   
2  Banne Buiksloot Buiksloot Buikslotermeer Flora...   
3  IJburg (Haveneiland - Rieteilanden - Steigerei...   
4  Admiralenbuurt Bos en Lommer (Kolenkitbuurt - ...   
5  Apollobuurt Buitenveldert Hoofddorppleinbuurt ...   
6  Bijlmer Bullewijk Driemond Gaasperdam Holend

In [11]:
# Create Neighborhood data frame - ne_df
ne_df = pd.DataFrame(table[3])
print(ne_df)

                       vteNeighborhoods of Amsterdam  \
0                                            Centrum   
1                                         Nieuw-West   
2                                              Noord   
3                                               Oost   
4                                               West   
5                                               Zuid   
6                                           Zuidoost   
7                                          Westpoort   
8  Former boroughs: De Baarsjes - Geuzenveld-Slot...   

                     vteNeighborhoods of Amsterdam.1  \
0  Binnenstad (Oude Zijde - Nieuwe Zijde) Grachte...   
1  Geuzenveld (De Eendracht) Nieuw Sloten Oostoev...   
2  Banne Buiksloot Buiksloot Buikslotermeer Flora...   
3  IJburg (Haveneiland - Rieteilanden - Steigerei...   
4  Admiralenbuurt Bos en Lommer (Kolenkitbuurt - ...   
5  Apollobuurt Buitenveldert Hoofddorppleinbuurt ...   
6  Bijlmer Bullewijk Driemond Gaasperdam Holend

In [12]:
# Dropping unnecessary rows & columns and rename
ne_df = ne_df.drop(['vteNeighborhoods of Amsterdam.2'],1)
ne_df = ne_df.drop(ne_df.index[8])
ne_df.rename(columns={'vteNeighborhoods of Amsterdam.1':'Neighborhood',
                      'vteNeighborhoods of Amsterdam':'Borough'}, inplace=True)

ne_df.to_csv('Neighborhoods.csv')
ne_df.head(10)

,Borough,Neighborhood
0,Centrum,Binnenstad (Oude Zijde - Nieuwe Zijde) Grachte...
1,Nieuw-West,Geuzenveld (De Eendracht) Nieuw Sloten Oostoev...
2,Noord,Banne Buiksloot Buiksloot Buikslotermeer Flora...
3,Oost,IJburg (Haveneiland - Rieteilanden - Steigerei...
4,West,Admiralenbuurt Bos en Lommer (Kolenkitbuurt - ...
5,Zuid,Apollobuurt Buitenveldert Hoofddorppleinbuurt ...
6,Zuidoost,Bijlmer Bullewijk Driemond Gaasperdam Holendre...
7,Westpoort,Teleport Westelijk Havengebied (Ruigoord)


In [13]:
# Merging the Borough and Neighborhood tables on "Borough" as new ams_df table containing Boroughs, Neighborhood and Population
ams_df = pd.merge(bor_df, ne_df, on='Borough')

In [14]:
print(ams_df)
ams_df.to_csv('amsterdam_loc.csv', index=False)

      Borough  Population                                     Neighbourhoods  \
0     Centrum       86422  Binnenstad, Grachtengordel, Haarlemmerbuurt, J...   
1       Noord       94766  Banne Buiksloot, Buiksloot, Buikslotermeer, Fl...   
2  Nieuw-West      151677  Geuzenveld, Nieuw Sloten, Oostoever, Osdorp, O...   
3        Oost      135767  IJburg, Indische Buurt, Eastern Docklands, Oud...   
4        West      143842  Frederik Hendrikbuurt, Houthaven, Spaarndammer...   
5   Westpoort         192                                          Westpoort   
6        Zuid      144432  Apollobuurt, Buitenveldert, Hoofddorppleinbuur...   
7    Zuidoost       87854    Bijlmermeer, Venserpolder, Gaasperdam, Driemond   

                                        Neighborhood  
0  Binnenstad (Oude Zijde - Nieuwe Zijde) Grachte...  
1  Banne Buiksloot Buiksloot Buikslotermeer Flora...  
2  Geuzenveld (De Eendracht) Nieuw Sloten Oostoev...  
3  IJburg (Haveneiland - Rieteilanden - Steigerei...  
4  A

**Decisions after inspecting the data:**
- As Amsterdam is a small city we notice that a segmentation based on Neighborhood does not add extra value to the analysis, so we keep Boroughs as baseline, adding the geo-coordinates.
- We will also drop Westport borough as this is the port and industrial area of Amsterdam (only 192 persons are living there).

In [15]:
# Drop neighborhood from ams_df
ams_df = ams_df.drop(['Neighborhood'], 1)
ams_df = ams_df.drop(ams_df.index[5])  # removing row index 5

In [16]:
# Loading data downloaded from data.asmterdam.nl with geo-coordinates in new amsgeo_df
amsgeo_df = pd.read_csv('ams_locations_nn.csv', sep=',')

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (23,43) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [18]:
amsgeo_df = amsgeo_df.drop(['Huisnummertoevoeging','Code stadsdeel','Code gebiedsgerichtwerkengebied','Naam gebiedsgerichtwerkengebied',
                            'Naam Wijk','Code Wijk','Naam buurt','Code buurt','Code bouwblok','X-coordinaat (RD)','Y-coordinaat (RD)','Type adres',
                            'Gebruiksdoel','Gebruiksdoel woonfunctie','Gebruiksdoel gezondheidszorgfunctie','Aantal eenheden complex',
                            'Soort object (feitelijk gebruik)','Aantal kamers','Toegang','Verdieping toegang','Aantal bouwlagen','Hoogste bouwlaag',
                            'Laagste bouwlaag','Oppervlakte (m2)','Oorspronkelijk bouwjaar','Eigendomsverhouding','Naam pand',
                            'Type woonobject','Ligging','Objecttype','Verblijfsobjectstatus','Geconstateerd','In Onderzoek',
                            'Openbareruimte-identificatie','Pandidentificatie','Verblijfsobjectidentificatie','Ligplaatsidentificatie',
                            'Standplaatsidentificatie','Nummeraanduidingidentificatie','Unnamed: 0','Naam openbare ruimte','Huisnummer','Huisletter',
                            'Postcode','Woonplaats'], 1)
amsgeo_df.head()

,Borough,Latitude (WGS84),Longitude (WGS84)
0,Oost,"52,3559240","4,9237822"
1,Zuid,"52,3433178","4,9066416"
2,West,"52,3642593","4,8745263"
3,Oost,"52,3561249","4,9246021"
4,Centrum,"52,3615781","4,8907208"


In [22]:
# Compiling final "amsdata_df" holding Borough, Pupulation and Geo-Coordinates information
amsdata_df = pd.merge(ams_df, amsgeo_df, on='Borough')

In [23]:
# Display and save "amsdata_df"
amsdata_df.to_csv('amsdata_df.csv', index=False)
amsdata_df.head()

,Borough,Population,Neighbourhoods,Latitude (WGS84),Longitude (WGS84)
0,Centrum,86422,"Binnenstad, Grachtengordel, Haarlemmerbuurt, J...","52,3615781","4,8907208"
1,Centrum,86422,"Binnenstad, Grachtengordel, Haarlemmerbuurt, J...","52,3618730","4,8903211"
2,Centrum,86422,"Binnenstad, Grachtengordel, Haarlemmerbuurt, J...","52,3593809","4,8897325"
3,Centrum,86422,"Binnenstad, Grachtengordel, Haarlemmerbuurt, J...","52,3594000","4,8900112"
4,Centrum,86422,"Binnenstad, Grachtengordel, Haarlemmerbuurt, J...","52,3777255","4,8814053"


In [24]:
# Dropping unnecessary rows & columns and rename
amsdata_df.rename(columns={'Latitude (WGS84)':'Latitude', 'Longitude (WGS84)':'Longitude'}, inplace=True)
amsdata_df.head()

,Borough,Population,Neighbourhoods,Latitude,Longitude
0,Centrum,86422,"Binnenstad, Grachtengordel, Haarlemmerbuurt, J...","52,3615781","4,8907208"
1,Centrum,86422,"Binnenstad, Grachtengordel, Haarlemmerbuurt, J...","52,3618730","4,8903211"
2,Centrum,86422,"Binnenstad, Grachtengordel, Haarlemmerbuurt, J...","52,3593809","4,8897325"
3,Centrum,86422,"Binnenstad, Grachtengordel, Haarlemmerbuurt, J...","52,3594000","4,8900112"
4,Centrum,86422,"Binnenstad, Grachtengordel, Haarlemmerbuurt, J...","52,3777255","4,8814053"


In [25]:
CLIENT_ID = 'ASGWICHNKBWGV3YPDCE3PCDTYKG5MWRLH51ZFXRQ0TZYWOUN' # Foursquare ID
CLIENT_SECRET = 'H5R5ADLASHKYVAEUOY4FAT314YKUDW2MULDUAXR5WCJMTXGC' # Foursquare Secret
VERSION = '20210516'
LIMIT = 30

In [26]:
address = 'Amsterdam, NL'

geolocator = Nominatim(user_agent="Foursquare_agent")
location = geolocator.geocode(address)
ams_latitude = location.latitude
ams_longitude = location.longitude
print('Amsterdam Geographical coordinates are Lat={}, Lng={} '.format(ams_latitude, ams_longitude))

Amsterdam Geographical coordinates are Lat=52.3727598, Lng=4.8936041 


In [27]:
amsdata_df.tail()

,Borough,Population,Neighbourhoods,Latitude,Longitude
531264,Zuidoost,87854,"Bijlmermeer, Venserpolder, Gaasperdam, Driemond","52,3071879","4,9460298"
531265,Zuidoost,87854,"Bijlmermeer, Venserpolder, Gaasperdam, Driemond","52,3072220","4,9461808"
531266,Zuidoost,87854,"Bijlmermeer, Venserpolder, Gaasperdam, Driemond","52,3072309","4,9459327"
531267,Zuidoost,87854,"Bijlmermeer, Venserpolder, Gaasperdam, Driemond","52,3073366","4,9462184"
531268,Zuidoost,87854,"Bijlmermeer, Venserpolder, Gaasperdam, Driemond","52,3073600","4,9461247"


In [28]:
def getNearbyVenues (names, latitude, longitude, radius=2000, limit=100, query='Restaurant'):
    
    venue_lst=[] # initialize empty list
    
    for name, lat, lng in zip (names, latitude, longitude):
        print(name)
        
    
        # Foursuare API explore generated per Neighbourhood lat / lng passed from toronto_df
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            query)
        
        # Get request
        results = requests.get(url).json()["response"]['groups'][0]['items']
                
        # return only relevant information for each nearby venue in the preliminary list
        venue_lst.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    venue_df = pd.DataFrame([item for venue_lst in venue_lst for item in venue_lst])
    venue_df.columns = ['Borough', 
                  'Borough Lat', 
                  'Borough Lng', 
                  'Venue', 
                  'Venue Lat', 
                  'Venue Lng', 
                  'Venue Category']

    print('{} venues were returned by Foursquare.'.format(venue_df.shape[0]))
    
    return (venue_df)

In [ ]:
amsterdam_venues_df = getNearbyVenues (names=amsdata_df['Borough'], 
                                     latitude=amsdata_df['Latitude'], 
                                     longitude=amsdata_df['Longitude'])

In [ ]:
# checking amsterdam_venues dataframe size and summary content
print(amsterdam_venues_df.shape)
amsterdam_venues_df.head(10)

In [ ]:
# Optional: Saving amsterdam_venues_df
amsterdam_venues_df.to_csv('amsterdam_venues.csv', index=False)

In [ ]:
print('There are {} uniques food serving categories.'.format(len(amsterdam_venues_df['Venue Category'].unique())))

In [ ]:
# Class distributions for 'Venue Category'

plt.figure(figsize=(15,20))
sns.countplot (y='Venue Category', data=amsterdam_venues_df)

plt.show()

In [ ]:
amsterdam_venues_df.groupby('Borough').count()

In [ ]:
# groupby 'Borough' and 'Venue Category' .size()
amsterdam_venues_df.groupby(['Borough','Venue Category']).size()

In [ ]:
amsterdam_rest_df = amsterdam_venues_df[amsterdam_venues_df['Venue Category'].str.contains('Restaurant')]

In [ ]:
# checking amsterdam_rest dataframe size and summary content
print(amsterdam_rest_df.shape)
amsterdam_rest_df.head(10)

In [ ]:
# Optional: Saving amsterdam_rest_df
amsterdam_rest_df.to_csv('amsterdam_rest.csv', index=False)

In [ ]:
amsterdam_rest_map = folium.Map(location=[ams_latitude, ams_longitude], zoom_start=12, control_scale = True)

# Toronto map markers (tag's)
for lat, lng, Borough, venue in zip(amsterdam_rest_df['Venue Lat'], amsterdam_rest_df['Venue Lng'], amsterdam_rest_df['Borough'], amsterdam_rest_df['Venue']):
    tag = '{}'.format(Borough)
    tag = folium.Popup(tag, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=1,
        popup=tag,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(amsterdam_rest_map)

# adding a folium feature allowing location, and coordinates shown with mouse-over and 
# mouse click on the map 
amsterdam_rest_map.add_child(folium.LatLngPopup())
    
amsterdam_rest_map

In [ ]:
amsterdam_rest_df.groupby('Borough').count()

In [ ]:
# Class distributions for Borough and Number of Restaurants
plt.figure(figsize=(10,4))
sns.countplot (y='Borough', data=amsterdam_rest_df)
plt.xlabel('Number of Restaurants')

plt.show()

In [ ]:
# Overall Class distribution for 'Venue Category' for Nieuw-West Borough

plt.figure(figsize=(10,6))
sns.countplot (y='Venue Category', data=amsterdam_rest_df[amsterdam_rest_df['Borough'].str.contains('Nieuw-West')])
plt.xlabel('Nieuw_West')
plt.show()

In [ ]:
# Overall Class distribution for 'Venue Category' in all amsterdam_rest_df dataset
plt.figure(figsize=(10,15))
sns.countplot (y='Venue Category', data=amsterdam_rest_df)
plt.xlabel('Number of Restaurants')
plt.show()

In [ ]:
# groupby 'Borough' and 'Venue Category' .size()
amsterdam_rest_df.groupby(['Borough','Venue Category']).size()

In [ ]:
print('There are {} uniques Restarant categories.'.format(len(amsterdam_rest_df['Venue Category'].unique())))

In [ ]:
# one hot encoding of the Venue Category
amsterdam_onehot = pd.get_dummies(amsterdam_rest_df[['Venue Category']], prefix="", prefix_sep="")

# add borough column back to dataframe
amsterdam_onehot['Borough'] = amsterdam_rest_df['Borough'] 

# move borough column to the first column
fixed_columns = [amsterdam_onehot.columns[-1]] + list(amsterdam_onehot.columns[:-1])
amsterdam_onehot = amsterdam_onehot[fixed_columns]

amsterdam_onehot.head()

In [ ]:
# Number of venues encoded
print('{} venues encoded.'.format(amsterdam_onehot.shape[0]-1))
# Shape of encoded dataframe
print('Shape amsterdam_onehot:', amsterdam_onehot.shape)

In [ ]:
# Optional - save the encoded df
amsterdam_onehot.to_csv('amsterdam_onehot.csv', index=False)

In [ ]:
amsterdam_grouped = amsterdam_onehot.groupby('Borough').mean().reset_index()

# Display the grouped dataframe
amsterdam_grouped

In [ ]:
amsterdam_grouped.groupby(['Borough','Restaurant']).sum()

In [ ]:
num_top_venues = 10

for place in amsterdam_grouped['Borough']:
    print("----"+place+"----")
    temp = amsterdam_grouped[amsterdam_grouped['Borough'] == place].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    # print(temp)
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
boroughs_venues_sorted = pd.DataFrame(columns=columns)
boroughs_venues_sorted['Borough'] = amsterdam_grouped['Borough']

for ind in np.arange(amsterdam_grouped.shape[0]):
    boroughs_venues_sorted.iloc[ind, 1:] = return_most_common_venues(amsterdam_grouped.iloc[ind, :], num_top_venues)

boroughs_venues_sorted

In [ ]:
# preparing the data for clustering - dropping the Borough column as it is not required
amsterdam_grouped_clustering = amsterdam_grouped.drop('Borough', 1)
amsterdam_grouped_clustering

In [ ]:
# Identify optiomal k using inertia. 
# We test range 1 to K clusters

inertias = [] 
mapping = {} 
K = range(1,7) 

for k in K: 
    #Building and fitting the model 
    kmeanModel = KMeans(n_clusters=k, n_init=50, max_iter=600, tol=0.0001, random_state=0)
    kmeanModel.fit(amsterdam_grouped_clustering)

    inertias.append(kmeanModel.inertia_) 

    # Inertia
    mapping[k] = kmeanModel.inertia_

In [ ]:
for key,val in mapping.items(): 
    print(str(key)+' : '+str(val))

In [ ]:
# plt.figure(figsize=(20,10))
plt.plot(K, inertias, 'bx-') 
plt.xlabel('Values of K') 
plt.ylabel('Inertia') 
plt.title('The Elbow Method using Inertia') 
plt.show()

In [ ]:
# set the number of clusters
kclusters = 3

# amsterdam_grouped_clustering = amsterdam_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, n_init=50, max_iter=600, tol=0.0001, random_state=0)

kmeans.fit(amsterdam_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20]

In [ ]:
# add clustering labels
boroughs_venues_sorted.insert(0, 'Cluster_Labels', kmeans.labels_)

amsterdam_merged = amsdata_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
amsterdam_merged = amsterdam_merged.join(boroughs_venues_sorted.set_index('Borough'), on='Borough')

amsterdam_merged.head() # check the last columns!

In [ ]:
# create map
map_clusters = folium.Map(location=[ams_latitude, ams_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(amsterdam_merged['Latitude'], amsterdam_merged['Longitude'], amsterdam_merged['Borough'], amsterdam_merged['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

for lat, lng, Borough, venue in zip(amsterdam_rest_df['Venue Lat'], amsterdam_rest_df['Venue Lng'], amsterdam_rest_df['Borough'], amsterdam_rest_df['Venue']):
    tag = '{}'.format(Borough)
    tag = folium.Popup(tag, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=0.5,
        popup=tag,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_clusters)
       
map_clusters